# Two-Layer Network Linear Regression
## Input the data

In [3]:
import random
import torch
from torch import nn, optim
import math
import resources as res
from IPython import display
from matplotlib import pyplot as plt

In [4]:
from res.plot_lib import plot_data, plot_model, set_default

ModuleNotFoundError: No module named 'res'

In [5]:
set_default()

NameError: name 'set_default' is not defined

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
seed = 1
random.seed(seed)
torch.manual_seed(seed)
N = 1000  # num_samples_per_class
D = 4  # dimensions
C = 1  # num_classes
H = 98  # num_hidden_units

In [8]:
import numpy as np

filename = r"C:\Users\Chun-Po\Desktop\NiMH 論文Code\Machine Learning\No Pre-screened Data\20211221_Random EIS and DCD35 Test/Variables_Modified.csv"
data = np.genfromtxt(filename, delimiter = ',', dtype = None, skip_header = 0, usecols = [7,15,37,38,53], encoding = 'UTF-8')
[Row,Column] = data.shape
print(data.shape)
print(data)
print(type(data[1,0]))

(99, 5)
[['V15min_DCD35' 'Log(VDT)_DCD35' 'D_Slope_EIS_C35_2hr'
  'D_Length_EIS_C35_2hr' 'DIS_Time_PVD']
 ['8.0099' '2.017033339' '45.04370294' '0.001329371' '2508']
 ['8.0158' '2.033423755' '46.49269343' '0.001349312' '2404']
 ['8.0094' '2.017033339' '46.63227144' '0.001310366' '2427']
 ['8.0185' '2.064457989' '47.67535742' '0.001376649' '2351']
 ['8.0253' '2.079181246' '49.08920812' '0.001395302' '2293']
 ['8.0247' '2.033423755' '49.19408916' '0.001389582' '2274']
 ['8.0365' '1.913813852' '49.22753394' '0.001421974' '2276']
 ['8.0308' '2.113943352' '49.8306218' '0.001408497' '2259']
 ['8.0669' '1.880813592' '50.55098553' '0.001462548' '2230']
 ['8.067' '1.662757832' '49.9902542' '0.001460373' '2195']
 ['8.0429' '2.133538908' '50.43523047' '0.001411743' '2240']
 ['8.1254' '1.255272505' '50.63156751' '0.001527578' '1907']
 ['8.1797' '1.342422681' '50.46905573' '0.001585527' '1745']
 ['8.5885' '2.008600172' '51.66275262' '0.001722713' '1382']
 ['8.6685' '1.146128036' '53.14539744' '0.00

In [10]:
#X = torch.unsqueeze(torch.linspace(-1, 1, 100), dim=1).to(device)
#y = X.pow(3) + 0.3 * torch.rand(X.size()).to(device)
X = torch.from_numpy(data[1:99,0:4].astype(np.float64))
y = torch.from_numpy(data[1:99,4:5].astype(np.float64))
print(X)
print(y)

tensor([[8.0099e+00, 2.0170e+00, 4.5044e+01, 1.3294e-03],
        [8.0158e+00, 2.0334e+00, 4.6493e+01, 1.3493e-03],
        [8.0094e+00, 2.0170e+00, 4.6632e+01, 1.3104e-03],
        [8.0185e+00, 2.0645e+00, 4.7675e+01, 1.3766e-03],
        [8.0253e+00, 2.0792e+00, 4.9089e+01, 1.3953e-03],
        [8.0247e+00, 2.0334e+00, 4.9194e+01, 1.3896e-03],
        [8.0365e+00, 1.9138e+00, 4.9228e+01, 1.4220e-03],
        [8.0308e+00, 2.1139e+00, 4.9831e+01, 1.4085e-03],
        [8.0669e+00, 1.8808e+00, 5.0551e+01, 1.4625e-03],
        [8.0670e+00, 1.6628e+00, 4.9990e+01, 1.4604e-03],
        [8.0429e+00, 2.1335e+00, 5.0435e+01, 1.4117e-03],
        [8.1254e+00, 1.2553e+00, 5.0632e+01, 1.5276e-03],
        [8.1797e+00, 1.3424e+00, 5.0469e+01, 1.5855e-03],
        [8.5885e+00, 2.0086e+00, 5.1663e+01, 1.7227e-03],
        [8.6685e+00, 1.1461e+00, 5.3145e+01, 1.8580e-03],
        [8.3596e+00, 1.0792e+00, 5.1369e+01, 1.7131e-03],
        [8.2799e+00, 1.1461e+00, 5.2030e+01, 1.6526e-03],
        [8.892

In [9]:
print("Shapes:")
print("X:", tuple(X.size()))
print(type(X))
print(X)
print("y:", tuple(y.size()))

Shapes:
X: (98, 4)
<class 'torch.Tensor'>
tensor([[8.0099e+00, 2.0170e+00, 4.5044e+01, 1.3294e-03],
        [8.0158e+00, 2.0334e+00, 4.6493e+01, 1.3493e-03],
        [8.0094e+00, 2.0170e+00, 4.6632e+01, 1.3104e-03],
        [8.0185e+00, 2.0645e+00, 4.7675e+01, 1.3766e-03],
        [8.0253e+00, 2.0792e+00, 4.9089e+01, 1.3953e-03],
        [8.0247e+00, 2.0334e+00, 4.9194e+01, 1.3896e-03],
        [8.0365e+00, 1.9138e+00, 4.9228e+01, 1.4220e-03],
        [8.0308e+00, 2.1139e+00, 4.9831e+01, 1.4085e-03],
        [8.0669e+00, 1.8808e+00, 5.0551e+01, 1.4625e-03],
        [8.0670e+00, 1.6628e+00, 4.9990e+01, 1.4604e-03],
        [8.0429e+00, 2.1335e+00, 5.0435e+01, 1.4117e-03],
        [8.1254e+00, 1.2553e+00, 5.0632e+01, 1.5276e-03],
        [8.1797e+00, 1.3424e+00, 5.0469e+01, 1.5855e-03],
        [8.5885e+00, 2.0086e+00, 5.1663e+01, 1.7227e-03],
        [8.6685e+00, 1.1461e+00, 5.3145e+01, 1.8580e-03],
        [8.3596e+00, 1.0792e+00, 5.1369e+01, 1.7131e-03],
        [8.2799e+00, 1.1461e+0

In [ ]:
plt.scatter(X.cpu().numpy(), y.cpu().numpy())
#plt.axis('equal');

## Linear model

In [1]:
learning_rate = 9e-3
lambda_l2 = 9e-5

In [13]:
# nn package to create our linear model
# each Linear module has a weight and bias
model = nn.Sequential(
    nn.Linear(D, H),
    nn.Linear(H, C)
)
model.to(device) # Convert to CUDA

# nn package also has different loss functions.
# we use MSE loss for our regression task
criterion = torch.nn.MSELoss()

# we use the optim package to apply
# stochastic gradient descent for our parameter updates
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=lambda_l2) # built-in L2

# Training
for t in range(40000):
  # Feed forward to get the logits
    y_pred = model(X.float())
  # Compute the loss (MSE)
    loss = torch.sqrt(criterion(y_pred.float(), y.float()))
    print("[EPOCH]: %i, [LOSS or RMSE]: %.6f" % (t, loss.item()))
    display.clear_output(wait=True)
  # zero the gradients before running
  # the backward pass.
    optimizer.zero_grad()
  # Backward pass to compute the gradient
  # of loss w.r.t our learnable params. 
    loss.backward()
  # Update params
    optimizer.step()

[EPOCH]: 39999, [LOSS or RMSE]: 1028.110229


In [14]:
# Plot trained model
print(model)

Sequential(
  (0): Linear(in_features=4, out_features=98, bias=True)
  (1): Linear(in_features=98, out_features=1, bias=True)
)


In [ ]:
plt.scatter(X.data.cpu().numpy(), y.data.cpu().numpy())
plt.plot(X.data.cpu().numpy(), y_pred.data.cpu().numpy(), 'r-', lw=5)
#plt.axis('equal');

## Two-layered network

In [1]:
learning_rate = 6e-2
lambda_l2 = 9e-5

In [11]:
# Number of networks
n_networks = 1
models = list()
y_pretrain = list()

# nn package also has different loss functions.
# we use MSE for a regression task
criterion = torch.nn.MSELoss()

for mod in range(n_networks):
    # nn package to create our linear model
    # each Linear module has a weight and bias
    model = nn.Sequential(
        nn.Linear(D, H),
        nn.ReLU(), #if mod < n_networks // 2 else nn.Tanh(),
        nn.Linear(H, C)
    )
    model.to(device)
    
    # Append models
    models.append(model)

    # we use the optim package to apply
    # ADAM for our parameter updates
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=lambda_l2) # built-in L2

    # e = 1.  # plotting purpose

    # Training
    for t in range(4000):

        # Feed forward to get the logits
        y_pred = model(X.float())
        
        # Append pre-train output
        if t == 0:
            y_pretrain.append(y_pred.detach())

        # Compute the loss and accuracy
        loss = torch.sqrt(criterion(y_pred.float(), y.float()))
        print(f"[MODEL]: {mod + 1}, [EPOCH]: {t}, [LOSS]: {loss.item():.6f}")
        display.clear_output(wait=True)

        # zero the gradients before running
        # the backward pass.
        optimizer.zero_grad()

        # Backward pass to compute the gradient
        # of loss w.r.t our learnable params. 
        loss.backward()

        # Update params
        optimizer.step()

[MODEL]: 1, [EPOCH]: 3999, [LOSS]: 96.181358


In [12]:
print(models[0], models[-1])

Sequential(
  (0): Linear(in_features=4, out_features=98, bias=True)
  (1): ReLU()
  (2): Linear(in_features=98, out_features=1, bias=True)
) Sequential(
  (0): Linear(in_features=4, out_features=98, bias=True)
  (1): ReLU()
  (2): Linear(in_features=98, out_features=1, bias=True)
)


## Predictions: Before Training

In [ ]:
for y_pretrain_idx in y_pretrain:
  # New X that ranges from -5 to 5 instead of -1 to 1
    X_new = torch.from_numpy(data[1:99,0:1].astype(np.float64))
        
    plt.plot(X_new.numpy(), y_pretrain_idx.cpu().numpy(), 'r-', lw=1)

plt.scatter(X.cpu().numpy(), y.cpu().numpy(), label='data')
# plt.axis('square')
# plt.axis((-1.1, 1.1, -1.1, 1.1));
y_combo = torch.stack(y_pretrain)
# plt.plot(X_new.numpy(), y_combo.var(dim=0).cpu().numpy(), 'g', label='variance');
plt.legend()

## Predictions: After Training

In [ ]:
y_pred = list()
relu_models = models[:n_networks // 2]
tanh_models = models[n_networks // 2:]
plt.figure(figsize=(20, 10))

def dense_prediction(models, non_linearity, zoom):
    plt.subplot(1, 2, 1 if non_linearity == 'ReLU' else 2)
    for model in models:
        # New X that ranges from -5 to 5 instead of -1 to 1
        X_new = torch.from_numpy(data[1:99,0:1].astype(np.float64))

        # Getting predictions from input
        with torch.no_grad():
            y_pred.append(model(X_new.float()))

        plt.plot(X_new.cpu().numpy(), y_pred[-1].cpu().numpy(), 'r-', lw=1)
    plt.scatter(X.cpu().numpy(), y.cpu().numpy(), label='data')
    #plt.axis('square')
    plt.axis(torch.tensor((8.0, 8.8, 0, 2500)) * zoom);
    y_combo = torch.stack(y_pred)
    plt.plot(X_new.cpu().numpy(), 10 * y_combo.var(dim=0).cpu().sqrt().numpy(), 'y', label='10 × std')
    plt.plot(X_new.cpu().numpy(), 10 * y_combo.var(dim=0).cpu().numpy(), 'g', label='30 × variance')
    plt.legend()
    plt.title(non_linearity + ' models')

z = 1  # try 1 or 4
dense_prediction(relu_models, 'ReLU', zoom=z)
dense_prediction(tanh_models, 'Tanh', zoom=z)